In [37]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import joblib
import sasc.config
import numpy as np
import sasc.viz
from sasc import analyze_helper
from sasc.config import FMRI_DIR, RESULTS_DIR
import dvu
dvu.set_style()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load pilot pickle

In [38]:
# pilot_name = 'pilot_story_data.pkl'
pilot_name = 'pilot3_story_data.pkl'

stories_data_dict = joblib.load(
    join(sasc.config.RESULTS_DIR, pilot_name))
if pilot_name == 'pilot_story_data.pkl':
    pilot_data_dir = '/home/chansingh/mntv1/deep-fMRI/story_data/20230504'
elif pilot_name == 'pilot2_story_data.pkl':
    pilot_data_dir = '/home/chansingh/mntv1/deep-fMRI/story_data/20230702'
elif pilot_name == 'pilot3_story_data.pkl':
    pilot_data_dir = '/home/chansingh/mntv1/deep-fMRI/story_data/20231106'


# load responses
default_story_idxs = np.where(
    np.array(stories_data_dict['story_setting']) == 'default')[0]
resp_np_files = [stories_data_dict['story_name_new'][i].replace('_resps', '')
                 for i in default_story_idxs]
resps_dict = {
    k: np.load(join(pilot_data_dir, k))
    for k in tqdm(resp_np_files)
}

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:06<00:00,  3.09s/it]


### Let's check the alignment

In [39]:
timings_list = stories_data_dict['timing']
story_names_list = list(resps_dict.keys())
resps = list(resps_dict.values())

In [40]:
TRIM = 5
for i in range(len(resps)):
    t = timings_list[i]
    duration_secs = t['time_running'].max()
    print(story_names_list[i], 'resp_length',
          resps[i].shape[0], 'story_trs',
          int(duration_secs // 2), 'story_length', duration_secs.round(1))  # , 'timings',
    diff = int(duration_secs // 2) - resps[i].shape[0]
    assert abs(diff - TRIM * 2) <= 1

GenStory12.npy resp_length 365 story_trs 374 story_length 749.0
GenStory13.npy resp_length 430 story_trs 439 story_length 878.7


### Let's check the alignment w/ paragraphs

In [62]:
def remove_repeated_words(paragraph):
    words = paragraph.split()
    new_words = [words[0]]
    for w in words[1:]:
        if w != new_words[-1]:
            new_words.append(w)
    return ' '.join(new_words)


# for story_num in [0]:
for story_num in [default_story_idxs]:
    rows = stories_data_dict["rows"][story_num]

    # get resp_chunks
    resp_story = resps_dict[
        stories_data_dict["story_name_new"][story_num].replace(
            '_resps', '')
    ].T  # (voxels, time)
    timing = stories_data_dict["timing"][story_num]
    paragraphs = stories_data_dict["story_text"][story_num].split("\n\n")
    if pilot_name in ['pilot3_story_data.pkl']:
        paragraphs = [remove_repeated_words(p) for p in paragraphs]
        assert len(paragraphs) == len(rows)
    resp_chunks = analyze_helper.get_resps_for_paragraphs(
        timing, paragraphs, resp_story, offset=2)
    assert len(resp_chunks) == len(paragraphs)

AssertionError: 

In [58]:
timing

,word,timing,time_running,word_len,ends_in_period,ends_in_comma
0,Once,0.710000,0.710000,4,False,False
1,upon,0.269606,0.979606,4,False,False
2,a,0.264278,1.243884,1,False,False
3,"time,",0.750000,1.993884,5,False,True
4,in,0.120000,2.113884,2,False,False
...,...,...,...,...,...,...
2732,thick,0.300000,748.316705,5,False,False
2733,and,0.210000,748.526705,3,False,False
2734,"thin,",0.130000,748.656705,5,False,True
2735,understanding,0.060000,748.716705,13,False,False


In [60]:
resp_story.shape

(95556, 365)

In [47]:
# for s in os.listdir('.'):
#     if os.path.isdir(s):
#         print(s)
#         for seed in os.listdir(s):
#             fname = os.path.join(s, seed, 'timings_processed.csv')
#             t = pd.read_csv(fname)
#             # remove rows with repeated word
#             t = t[t['word'] != t['word'].shift(1)]
#             t['time_running'] = np.cumsum(t["timing"])
#             t.to_csv(fname, index=False)


paragraphs = [remove_repeated_words(p) for p in paragraphs]

In [ ]:
paragraphs

In [43]:
start_indexes, end_indexes = analyze_helper.get_start_end_indexes_for_paragraphs(
    timing, paragraphs)
print(paragraphs[0].split())
print(timing['word'].values.tolist())
print(
    f'should have {len(paragraphs)} paragraphs but only found {len(resp_chunks)}')

AssertionError: (24, 'me', 'love')